## Importation des fichiers utiles

In [26]:
import urllib.request
import json
import pandas as pd
import numpy as np
from PIL import Image, ExifTags
from sklearn.cluster import KMeans


NOMBRE_IMAGE=141

<br>

## Demande de Json
Nous Faisons une Requète WikiData pour demander un Json contenant le lien menant à des libres d'une certaine catégorie. Nous avons essayé plusieurs types d'images différentes :

mythical creature = Q2239243<br>
photograph = Q125191<br>
flower = Q506

In [27]:
url ="https://query.wikidata.org/sparql?query=SELECT%20%3Fimage%20%3Fim%20WHERE%20%7B%0A%20%20%3Fimage%20wdt%3AP31%20wd%3AQ506.%0A%20%20%3Fimage%20wdt%3AP18%20%3Fim%0A%20%20%7D%0ALIMIT%20200&format=json"
response = urllib.request.urlopen(url)
myJson = json.loads(response.read().decode("utf-8"))
myJsonNormalized = pd.json_normalize(myJson)

<br>

## Télechargement des images ##
A partir du Json contenant les liens menant aux images, nous lancons des requètes URLLIB pour télecharger ces images.

In [30]:
#telechargement des images

responseJson = myJsonNormalized["results.bindings"]
for i in range(len(responseJson[0])):   
    imageLink = responseJson[0][i]["im"]["value"]
    urllib.request.urlretrieve(imageLink, "./images/image"+str(i)+".jpg")
    NOMBRE_IMAGE=i

<br>

## Récupération des métadonnées
L'obhectif de cette partie est de récupérer les informations des images, nous récupérons les images suivantes :<br>
1. Récupération Exif utiles
2. Récupération couleurs dominantes (K-means)
3. Récupération taille des images
4. Détermination portrait ou paysage

Apres cela, nous avons creer un Json **metaDataJson.json** contenant toutes les informationns precedentes

In [1]:
# Récupération des métadonnées

metaData ={}
for i in range(NOMBRE_IMAGE):
    im = "./images/image"+str(i)+".jpg"
    try:
        imgfile = Image.open(im).convert("RGB")

        #Recupération Exif utiles
        exif = {}
        exifCritere= ("Model","Make","ImageDescription","ExposureTime","GPSInfo","ISO","DateTimeOriginal")
        for k,v in  imgfile.getexif().items() :
            if k in ExifTags.TAGS :
                if ExifTags.TAGS[k] in exifCritere:
                    exif = {ExifTags.TAGS[k]:v}

        # Recupération couleurs dominantes (K-means)
        numarray = np.array(imgfile.getdata(), np.uint8)
        clusters  = KMeans(n_clusters=3)
        clusters.fit(numarray[::50])
        L = clusters.cluster_centers_.astype(int).tolist()

        # Recupération taille des images
        width = imgfile.width
        height = imgfile.height
        mode = imgfile.mode

        # Détermination portrait ou paysage
        if abs(1-width/height)<0.1 :
            orientation='carre' 
        elif width>height:
            orientation='paysage'
        else: 
            orientation="portrait"

        metaData["image"+str(i)] = { "width" : width, "height": height, "exif": exif, "mode":mode, "tags":[], "couleurs":L, "orientation":orientation }
    except Exception:
        print("l'image "+str(i) +" ne peut pas etre ouverte")
metaData

metaDatajson = json.dumps(metaData)
f = open("metaDataJson.json", "w")
f.write(metaDatajson)
f.close()



NameError: name 'NOMBRE_IMAGE' is not defined

## Analyse 

In [ ]:
def analyse(nameUser):
    jsonUser=json.loads("jsonUser.json")
    jsonUserNormalized = pd.json_normalize(jsonUser)
    L = jsonUserNormalized[nameUser]["image"]
    couleur1=np.array([0,0,0])
    couleur2=[]
    couleur3=[]
    orientationPref=[]
    for image in L:
        for j in range(3):
            couleur1+=np.square(np.array(jsonUserNormalized[image]['couleurs'][0][j]))
            couleur2+=np.square(np.array(jsonUserNormalized[image]['couleurs'][1]))
            couleur3+=np.square(np.array(jsonUserNormalized[image]['couleurs'][2]))
        
        orientationPref.append(jsonUserNormalized[image]['exif']['model'])

    NewColor = sqrt((R1^2+R2^2)/2),sqrt((G1^2+G2^2)/2),sqrt((B1^2+B2^2)/2)

In [ ]:
def visualisation(parametres,nameUser='ALL'):
    myJson = json.loads("metaDataJson.json")
    myJsonNormalized = pd.json_normalize(myJson)
    L=[]

    jsonUser=json.loads("jsonUser.json")
    jsonUserNormalized = pd.json_normalize(jsonUser)
    L = jsonUserNormalized[nameUser]["image"]
    yearData =[]
    modelData =[]
    for image in L:
        if 'year' in myJsonNormalized[image]['exif']:
            yearData.append(myJsonNormalized[image]['exif']['year'])
        if 'mode1' in myJsonNormalized[image]:
            modelData.append(myJsonNormalized[image]['exif']['model'])
    max(yearData, key=yearData.count)
    max(modelData, key=modelData.count)

